In [1]:
import pandas as pd
from utils.patterns import patterns_common_words, numeric_patterns, unit_patterns
import re
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)


lab = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\D1_BD_PADRIS_regicor\AP_Laboratoris.csv", sep = "|")

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_22680\357312902.py:8: DtypeWarning: Columns (3,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  lab = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\D1_BD_PADRIS_regicor\AP_Laboratoris.csv", sep = "|")


In [2]:
lab["lab_resultat"] = lab["lab_resultat"].fillna("nc")

In [3]:
lab

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max
0,10254,2010,17/08/2010,81705546520,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
1,7314,2011,01/08/2011,80100100846,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
2,8871,2008,17/03/2008,31774972,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
3,3241,2008,13/05/2008,51354073,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
4,11405,2010,20/04/2010,42005701677,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7825016,9255,2016,01/09/2016,90105546769,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2
7825017,10726,2016,04/11/2016,110400100272,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2
7825018,7691,2017,13/12/2017,121305613907,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2
7825019,11051,2017,04/10/2017,100405547798,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2


In [4]:
def clear_typos(df):
    """Initial data cleaning by removing typographical errors and extraneous characters from the result values."""
    
    # Apply the cleaning steps to the specified column
    df["clean_result"] = df["lab_resultat"].apply(lambda x: re.sub(r'[!#$&\'();?@_`{|}~"\[\]]', '', x))  # Remove special characters
    df["clean_result"] = df["clean_result"].apply(lambda x: re.sub(r'^=|=$', '', x))  # Remove leading and trailing equal signs
    df["clean_result"] = df["clean_result"].apply(lambda x: re.sub(r'^\s+|\s+$', '', x))  # Remove leading/trailing spaces

    return df
    

In [8]:
def process_adjacent(df, pattern, replacement, comment_flag):
    """ Process adjacent patterns in the 'clean_result' column of the DataFrame."""
    # Flagging records and updating 'cleaning_comments'
    mask = df['clean_result'].str.contains(pattern, flags=re.IGNORECASE, regex=True)
    df.loc[mask, 'cleaning_comments'] = comment_flag
    # Substituting the pattern in 'clean_result'
    df.loc[mask, 'clean_result'] = df.loc[mask, 'clean_result'].apply(lambda x: re.sub(pattern, replacement, str(x), flags=re.IGNORECASE))
    return df

def handle_extra_variables(lab_data, patterns_common_words, numeric_patterns, unit_patterns, report=True):
    # Initialize the cleaning_comments column if not already present
    if 'cleaning_comments' not in lab_data.columns:
        lab_data['cleaning_comments'] = pd.NA

    # Step 1: Handle flags (positive, negative, normal, abnormal)
    for flag, patterns in patterns_common_words.items():
        for pattern in patterns:
            # Apply the flags to clean_result based on the patterns in the dictionary
            lab_data['clean_result'] = lab_data['clean_result'].apply(
                lambda x: re.sub(pattern, flag, str(x), flags=re.IGNORECASE)
            )
    
    # Step 2: Handle flags and units adjacent to results
    adjacent_units1 = r'^(' + numeric_patterns['Qn1'] + r')\s*(' + numeric_patterns['units'] + r')$'
    adjacent_units2 = r'^(' + numeric_patterns['units'] + r')\s*(' + numeric_patterns['Qn1'] + r')$'
    
    # Flagging records with adjacent units before the result
    mask = lab_data['clean_result'].str.contains(adjacent_units1, flags=re.IGNORECASE, regex=True)
    lab_data.loc[mask, 'cleaning_comments'] = 'unit'
    lab_data.loc[mask, 'clean_result'] = lab_data.loc[mask, 'clean_result'].apply(lambda x: re.sub(pattern, r'\1', str(x), flags=re.IGNORECASE))

    # Flagging records with adjacent units after the result
    mask = lab_data['clean_result'].str.contains(adjacent_units2, flags=re.IGNORECASE, regex=True)
    lab_data.loc[mask, 'cleaning_comments'] = 'unit'
    lab_data.loc[mask, 'clean_result'] = lab_data.loc[mask, 'clean_result'].apply(lambda x: re.sub(pattern, r'\2', str(x), flags=re.IGNORECASE))

    # Removing negative signs from the result
    #lab_data['cleaning_comments'][lab_data['clean_result'].str.contains(r'^(-\s*)(' + numeric_patterns['Qn1'] + r')$', regex=True)] = 'flag'
    #lab_data['clean_result'] = lab_data['clean_result'].apply(lambda x: re.sub(r'^(-\s*)(' + numeric_patterns['Qn1'] + r')$', r'\2', str(x)))

    # Removing positive signs from the result
    #lab_data['cleaning_comments'][lab_data['clean_result'].str.contains(r'^[\+]\s*(' + numeric_patterns['Qn1'] + r')$', regex=True)] = 'flag'
    #lab_data['clean_result'] = lab_data['clean_result'].apply(lambda x: re.sub(r'^[\+]\s*(' + numeric_patterns['Qn1'] + r')$', r'\2', str(x)))

    # Handling percent, and exponents
    # Percent
    #lab_data['cleaning_comments'][lab_data['clean_result'].str.contains(r'^(' + numeric_patterns['Qn1'] + r') *%$', regex=True)] = lab_data['cleaning_comments'].apply(lambda x: 'Percent' if pd.isna(x) else x + ', Percent')
    #lab_data['clean_result'] = lab_data['clean_result'].apply(lambda x: re.sub(r'^(' + numeric_patterns['Qn1'] + r') *%$', r'\1', str(x)))
    
    # Exponent
    #lab_data['cleaning_comments'][lab_data['clean_result'].str.contains(r'^(' + numeric_patterns['Qn1'] + r')(' + numeric_patterns['exponent'] + r')$', regex=True)] = lab_data['cleaning_comments'].apply(lambda x: 'Exponents' if pd.isna(x) else x + ', Exponents')
    #lab_data['clean_result'] = lab_data['clean_result'].apply(lambda x: re.sub(r'^(' + numeric_patterns['Qn1'] + r')(' + numeric_patterns['exponent'] + r')$', r'\1', str(x)))
    
    # Remove `*` and `^` from exponents
    #lab_data['clean_result'] = lab_data['clean_result'].apply(lambda x: re.sub(r'[\*\^]', '', str(x)))
    
    # Report units
    if report:
        units_records = lab_data['cleaning_comments'].str.contains('flag|percent|exponents|units').sum()
        units_records_percent = (units_records / len(lab_data) * 100) if len(lab_data) > 0 else 0
        print(f"{units_records} records with units (%, exponents, or other units) -> units removed with cleaning comment 'percent, exponent, or units'.")
        print(f"These represent {units_records_percent:.3f}% of the total records.")
    
    return lab_data

In [9]:
test = clear_typos(lab)
test2 = handle_extra_variables(test, patterns_common_words, numeric_patterns, unit_patterns, report=True)

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_22680\1538951786.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = lab_data['clean_result'].str.contains(adjacent_units1, flags=re.IGNORECASE, regex=True)
C:\Users\mhuertas\AppData\Local\Temp\ipykernel_22680\1538951786.py:33: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = lab_data['clean_result'].str.contains(adjacent_units2, flags=re.IGNORECASE, regex=True)


0 records with units (%, exponents, or other units) -> units removed with cleaning comment 'percent, exponent, or units'.
These represent 0.000% of the total records.


In [7]:
test2[test2["cleaning_comments"] == "unit"]

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max,clean_result,cleaning_comments


In [10]:
numeric_patterns['units']

'^[a-zA-Z]{1,4}\\s?\\/\\s?[a-zA-Z]{1,4}$'